In [1]:
import numpy as np
import pandas as pd
from chromatography import *
from separation_utility import *
from torch import optim, tensor, Tensor
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mayavi import mlab
%matplotlib qt

In [2]:
alists = []
alists.append(pd.read_csv(f'../data/GilarSample.csv'))
alists.append(pd.read_csv(f'../data/Peterpeptides.csv'))
alists.append(pd.read_csv(f'../data/Roca.csv'))
alists.append(pd.read_csv(f'../data/Peter32.csv'))
alists.append(pd.read_csv(f'../data/Eosin.csv'))
alists.append(pd.read_csv(f'../data/Alizarin.csv'))
alists.append(pd.read_csv(f'../data/Controlmix2.csv'))


In [3]:
def loss_field(exp, taus, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], taus)
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

def average_over_equal_intervals(arr, interval):
    return np.mean(arr.reshape(-1, interval), axis=1)

In [ ]:
taus = [.25, .25, 10]
pol = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 100),
                nn.ELU(inplace=True),
                PermEqui2_max(100, 100),
                nn.ELU(inplace=True),
                PermEqui2_max(100, 30),
                nn.ELU(inplace=True),
            ),
            rho = nn.Sequential(
                nn.Linear(30, 60),
                nn.ELU(inplace=True),
                nn.Linear(60, 120),
                nn.ELU(inplace=True),
                Rho(n_steps=len(taus), hidden=100, in_dim=120, sigma_max=.3, sigma_min=.01),
            )
        )
losses = reinforce_gen(
    alists = alists, 
    policy = pol, 
    delta_taus = taus, 
    num_episodes = 200000, 
    batch_size = 10, 
    lr = .05, 
    optim = lambda a, b: torch.optim.SGD(a, b, momentum=0.65),
    lr_decay_factor= 0.5,
    lr_milestones=1000,
    print_every = 1000,
    baseline = .55,
    max_norm = 1.7,
    max_rand_analytes = 30,
    min_rand_analytes = 10,
    rand_prob = 0.7
)


Loss: 1.5827106322805464, epoch: 1000/200000


In [6]:
plt.plot(np.linspace(0, 200000, 2000),average_over_equal_intervals(losses[0], 100))
plt.title("Loss (average of 500 random sets)")
plt.xlabel("Episode")
plt.ylabel("Loss")
plt.show()

In [7]:
i = 0
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
mu, sig = pol(torch.Tensor(alists[i][['S', 'lnk0']].values))
exp.run_all(mu.detach().numpy(), taus)

exp.print_analytes(title=f"Solvent Strength Program(Gen)\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
L, B, mus, sigmas = reinforce_single_from_gen(
        alist=alists[i], 
        policy=pol, 
        delta_taus=taus, 
        num_episodes=3000, 
        batch_size=10, 
        lr=.1, 
        optim=torch.optim.SGD,
        lr_decay_factor=.5,
        lr_milestones=500,
        print_every=500,
        baseline=0.65,
        max_norm=1.5
    )

In [ ]:
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
exp.run_all(mus[-1,:], taus)
exp.print_analytes(title=f"Solvent Strength Program(Iso)\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
exp.run_all(B, taus)
exp.print_analytes(title=f"Best Solvent Strength Program\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
plt.plot(mus)

In [ ]:
plt.plot(sigmas)

In [ ]:
pol = PolicyGeneral(
            phi = nn.Sequential(
                PermEqui2_max(2, 3),
                nn.ELU(inplace=True),
                PermEqui2_max(3, 3),
                nn.ELU(inplace=True),
                PermEqui2_max(3, 3),
                nn.ELU(inplace=True),
            ),
            rho = RhoTime(n_steps=3, hidden=5, in_dim=3, sigma_max=.3, sigma_min=.02)
        )
losses = reinforce_delta_tau_gen(
    alists = alists, 
    policy = pol,
    num_episodes = 10000, 
    batch_size = 10, 
    lr = .1, 
    optim = lambda a, b: torch.optim.SGD(a, b),
    lr_decay_factor= 0.75,
    lr_milestones=1000,
    print_every = 100,
    baseline = .55,
    max_norm = 1.2,
    max_rand_analytes = 35,
    min_rand_analytes = 18,
    rand_prob = 0.7
)

In [ ]:
plt.plot(np.linspace(0, 200000, 200),average_over_equal_intervals(losses[0], 1000))
plt.title("Loss [variable delta tau] (average of 1000 random sets)")
plt.xlabel("Episode")
plt.ylabel("Loss")
plt.show()

In [ ]:
i = 6
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
mu, _ = pol(torch.Tensor(alists[i][['S', 'lnk0']].values))
mu = mu.tolist()
mu.append(10.)
exp.run_all(mu[0:3], mu[3:])

exp.print_analytes(title=f"Solvent Strength Program\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)

In [ ]:
L, B, mus, sigmas = reinforce_single_from_delta_tau_gen(
        alist=alists[i], 
        policy=pol,
        num_episodes=5000, 
        batch_size=10, 
        lr=.1, 
        optim=torch.optim.SGD,
        lr_decay_factor=.5,
        lr_milestones=500,
        print_every=500,
        baseline=0.65,
        max_norm=1.2
    )

In [ ]:
plt.plot(mus)

In [ ]:
plt.plot(L)

In [ ]:
exp = ExperimentAnalytes(k0 = alists[i].k0.values, S = alists[i].S.values, h=0.001,run_time=10.0)
mu = mus[-1].tolist()
mu.append(10.)
exp.run_all(mu[0:3], mu[3:])

exp.print_analytes(title=f"Solvent Strength Program\nLoss:{round(exp.loss(), 4)}", rc=(10,10), angle=40)